In [1]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colors import LogNorm
import datetime
import os
dp = '/home/ebert/jupyter-notebooks/FNode-data'
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]

exclude = [2]

In [2]:
exp_names = [
    '2018_07_11_09_29_44_trap-lifetime',
    '2018_07_11_13_45_13_trap-lifetime-2'
]

single_only = True

rets = 'retention'
errs = 'error'
if single_only:
    rets = 'conditional_retention'
    errs = 'conditional_retention_err'

q = qdp.QDP(base_data_path=dp)
for exp_name in exp_names:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
    q.load_data_file(experiment_file)

print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
cs_cuts = [[33],[3]]
rb_cuts = [[31],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds(exclude_rois=exclude, ncondition=1)
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"

q.experiments[0]['variable_list']
try:
    xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
except IndexError:
    xlab = 'iterator'

# tbl_str = "ivar:\t{0:.3f}\tloading:{1:.3f}\tretention:{2:.3f} +- {3:.3f}"
# for i in range(len(retention[rets].flatten())):
#     print(tbl_str.format(
#         retention['ivar'].flatten()[i],
#         retention['loading'].flatten()[i],
#         retention[rets].flatten()[i],
#         retention[errs].flatten()[i]
#     ))

Command '['git', 'describe', '--always']' returned non-zero exit status 128
data at: 2018_07_11/2018_07_11_09_29_44_trap-lifetime/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
data at: 2018_07_11/2018_07_11_13_45_13_trap-lifetime-2/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'gap_time']
ivar:	0.000	loading:0.286	retention:0.957 +- 0.024
ivar:	0.000	loading:0.348	retention:0.920 +- 0.024
ivar:	0.000	loading:13.955	retention:0.000 +- 0.024
ivar:	0.000	loading:0.302	retention:0.792 +- 0.041
ivar:	0.000	loading:0.314	retention:0.763 +- 0.041
ivar:	0.000	loading:0.000	retention:0.000 +- 0.041
ivar:	0.000	loading:0.280	retention:0.709 +- 0.048
ivar:	0.000	loading:0.264	retention:0.603 +- 0.048
ivar:	0.000	loading:0.000	retention:0.000 +- 0.048
ivar:	0.000	loading:0.354	retention:0.535 +- 0.046
ivar:	0.000	loading:0.258	retention:0.636 +- 0.046
ivar:	0.000	loading:0.000	retention:0.000 +- 0.046
ivar:	0.000	loading:

/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:131: RuntimeWarning: invalid value encountered in sqrt
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))


In [9]:
import scipy.optimize as optimize

fig, ax = plt.subplots(figsize=(2*3.375,3.5))
ret = []
err = []
fmt_map = ['bo', 'rd']
ivar = [
    np.delete(np.concatenate((np.linspace(8.5,1e4,6), np.linspace(1.2e4,2e4,5)))/1e3, -3),
    np.concatenate((np.linspace(8.5,1e4,6), np.linspace(1.2e4,2e4,5)))/1e3
]

def expon(x, a, tau, b):
    return b + a*np.exp(-x/tau)

for r in range(2):
    ret.append(np.concatenate((retention[rets][0,:,r], retention[rets][1,:-1,r])))
    err.append(np.concatenate((retention[errs][0,:,r], retention[errs][1,:-1,r])))
    if r == 0:
        ret[r] = np.delete(ret[r], -3)
        err[r] = np.delete(err[r], -3)
    guess = [1,10]
    popt, pcov = optimize.curve_fit(
        lambda t, a, tau: expon(t, a, tau, 0),
        ivar[r],
        ret[r],
        p0=guess,
        sigma=err[r]
    )
    print(popt)
    print(np.sqrt(np.diagonal(np.abs(pcov))))
    xs = np.linspace(0,21,100)
    ax.plot(xs, expon(xs, popt[0], popt[1], 0), color=marker_styles[r]['color'], linestyle='--')
    ax.errorbar(ivar[r], ret[r], yerr=err[r], **marker_styles[r])
        
ax.set_ylim(0., 1.0)
ax.set_xlim(-0.5, 21)
ax.set_xlabel('Time (s)', fontsize=14)
ax.set_ylabel('Retention', fontsize=14)
ax.grid(which='both')
plt.tight_layout()

fmts = ['pdf', 'png', 'eps']
out = '/home/ebert/jupyter-notebooks/FNode/analyses/CollisionPaper/trap-lifetime/'
fn = os.path.join(out, 'trap-lifetime.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

[  0.95692399  11.29052678]
[ 0.01630036  0.48585541]
[  0.91393715  13.4564238 ]
[ 0.02352775  0.83475513]
